In [ ]:
#!pip install requests shapely pyproj pandas folium

In [ ]:
import os, math, json, requests
import pandas as pd
from typing import List, Tuple, Dict
from shapely.geometry import shape, Polygon, MultiPolygon, Point, LineString, mapping
from shapely.ops import unary_union
from pyproj import Transformer, CRS
import folium
from google.colab import userdata

In [ ]:
# ----------------- CONFIG ----------------------------------------
ORIGIN_TEXT = "Salesforce Tower, San Francisco"
DEST_TEXT   = "Sotto Mare, San Francisco"
PROFILE     = "foot-walking"     # ORS profile (walking for scenic demos)
ISO_MIN     = 20                 # minutes for each isochrone
TOP_K       = 10                 # top waypoints to display
ETA_TOLERANCE_MIN = 10           # simple filtering to eliminate routes more than X minutes longer than fastest route
MAP_HTML    = "scenic_search_zone_map.html"

# Example free-text preference; try variants:
# "prefer waterfront and parks, maybe viewpoints; coffee would be nice"
PREFERENCE_TEXT = "Prefer waterfront and green parks; viewpoints would be great; coffee nearby."
CONSTRAINTS_TEXT = "prefer shortest; avoid stairs and ferries; walking"

os.environ["ORS_API_KEY"] = userdata.get('ORS_API_KEY')

In [ ]:
# ----------------- SIMPLE NLU: text -> feature weights ------------------------
# Can swap this with an LLM later; this is a keyword lexicon
FEATURE_LEXICON = {
    "waterfront": ["waterfront", "river", "lake", "bay", "harbor", "seaside", "coast", "canal"],
    "greenery":   ["green", "parks", "park", "greenway", "trees", "nature", "garden"],
    "viewpoint":  ["viewpoint", "scenic", "overlook", "vista", "lookout"],
    "cafe":       ["coffee", "café", "cafe", "espresso"],
}
DEFAULT_WEIGHTS = {"waterfront": 0.6, "greenery": 0.6, "viewpoint": 0.4, "cafe": 0.2}

def parse_preferences(text: str) -> Dict[str, float]:
    t = text.lower()
    weights = {k: 0.0 for k in DEFAULT_WEIGHTS.keys()}
    for feat, kws in FEATURE_LEXICON.items():
        if any(kw in t for kw in kws):
            weights[feat] = DEFAULT_WEIGHTS[feat]
    # if nothing matched, fall back to greenery (pleasant default)
    if sum(weights.values()) == 0.0:
        weights["greenery"] = DEFAULT_WEIGHTS["greenery"]
    return weights

def parse_routing_constraints(text: str):
    t = text.lower()
    profile = "foot-walking"
    if "driving" in t: profile = "driving-car"
    elif "cycling" in t or "bike" in t: profile = "cycling-regular"
    elif "walking" in t or "foot" in t: profile = "foot-walking"

    preference = "fastest"
    if "shortest" in t: preference = "shortest"

    # ORS-supported avoid_features (varies by profile; these are fairly standard)
    possible_avoids = {
        "ferries": ["ferry", "ferries"],
        "tunnels": ["tunnel", "tunnels"],
        "tollways": ["tolls", "tollway", "tollways"],
        "motorways": ["motorway", "motorways", "highway", "highways"],
        "steps": ["stairs", "steps"],        # useful for walking
        "fords": ["fords", "ford"]
    }
    avoid_features = []
    for feat, kws in possible_avoids.items():
        if any(kw in t for kw in kws):
            avoid_features.append(feat)

    return {
        "profile": profile,
        "preference": preference,
        "avoid_features": avoid_features  # pass through to ORS 'options.avoid_features'
    }

In [ ]:
# ----------------- HTTP setup -------------------------------------------------
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "scenic-routing-demo/1.1 (email@example.com)"})

In [ ]:
# ----------------- Geocode / Isochrones / Overpass ---------------------------
def geocode_nominatim(q: str) -> Tuple[float, float]:
    url = "https://nominatim.openstreetmap.org/search"
    r = SESSION.get(url, params={"q": q, "format": "json", "limit": 1}, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data:
        raise ValueError(f"Geocoding failed for: {q}")
    return float(data[0]["lat"]), float(data[0]["lon"])

def orservice_isochrone(lon: float, lat: float, minutes: int, profile: str) -> dict:
    url = f"https://api.openrouteservice.org/v2/isochrones/{profile}"
    key = os.environ.get("ORS_API_KEY")
    if not key:
        raise RuntimeError("Missing ORS_API_KEY env var.")
    payload = {
        "locations": [[lon, lat]],
        "range": [minutes * 60],
        "area_units": "m",
    }
    r = SESSION.post(url, headers={"Authorization": key}, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()

def isochrone_polygon(geojson: dict):
    geoms = [shape(f["geometry"]) for f in geojson.get("features", [])]
    if not geoms:
        raise ValueError("No geometry in isochrone response.")
    return unary_union(geoms)

def bbox_from_polygon(poly: Polygon | MultiPolygon) -> Tuple[float,float,float,float]:
    minx, miny, maxx, maxy = poly.bounds
    # Overpass bbox expects: south, west, north, east (lat,lon order)
    return (miny, minx, maxy, maxx)

def overpass(query: str) -> dict:
    url = "https://overpass-api.de/api/interpreter"
    r = SESSION.post(url, data={"data": query}, timeout=120)
    r.raise_for_status()
    return r.json()

In [ ]:
# ----------------- Feature queries ------------------------------
# We’ll pull polygons/lines for “land features” that help scenic scoring, and points for POIs.
def build_overpass_multi_query(bbox, weights: Dict[str, float]) -> Dict[str, str]:
    south, west, north, east = bbox
    parts = {}

    if weights.get("waterfront", 0) > 0:
        parts["water"] = f"""
          way["natural"="water"]({south},{west},{north},{east});
          relation["natural"="water"]({south},{west},{north},{east});
          way["waterway"~"river|canal"]({south},{west},{north},{east});
          relation["waterway"="riverbank"]({south},{west},{north},{east});
          way["natural"="coastline"]({south},{west},{north},{east});
        """

    if weights.get("greenery", 0) > 0:
        parts["parks"] = f"""
          way["leisure"~"park|garden|nature_reserve"]({south},{west},{north},{east});
          relation["leisure"~"park|garden|nature_reserve"]({south},{west},{north},{east});
          way["landuse"~"grass|forest"]({south},{west},{north},{east});
          relation["landuse"~"forest"]({south},{west},{north},{east});
        """

    if weights.get("viewpoint", 0) > 0:
        parts["viewpoints"] = f"""
          node["tourism"="viewpoint"]({south},{west},{north},{east});
          node["natural"="beach"]({south},{west},{north},{east});
          way["natural"="beach"]({south},{west},{north},{east});
          relation["natural"="beach"]({south},{west},{north},{east});
        """

    if weights.get("cafe", 0) > 0:
        parts["cafes"] = f"""
          node["amenity"="cafe"]({south},{west},{north},{east});
          node["amenity"="coffee_shop"]({south},{west},{north},{east});
        """

    queries = {}
    # Polygons/lines (land features) with geometry
    geom_parts = []
    for k in ("water", "parks"):
        if k in parts:
            geom_parts.append(parts[k])
    if geom_parts:
        queries["land_geoms"] = f"[out:json][timeout:60];({''.join(geom_parts)});out geom;"

    # Points (waypoints/POIs) with center
    point_parts = []
    for k in ("viewpoints", "cafes"):
        if k in parts:
            point_parts.append(parts[k])
    if point_parts:
        queries["poi_points"] = f"[out:json][timeout:60];({''.join(point_parts)});out center;"

    return queries

def elements_to_geoms(elems: List[dict]):
    polys_lines, points = [], []
    for e in elems:
        if "geometry" in e:
            coords = [(pt["lon"], pt["lat"]) for pt in e["geometry"]]
            if len(coords) > 2 and coords[0] == coords[-1]:
                polys_lines.append(Polygon(coords))
            else:
                polys_lines.append(LineString(coords))
        elif "center" in e:
            points.append(Point(e["center"]["lon"], e["center"]["lat"]))
        elif e.get("type") == "node" and "lon" in e and "lat" in e:
            points.append(Point(e["lon"], e["lat"]))
    return polys_lines, points

In [ ]:
# ----------------- Scoring --------------------------------------
def local_metric_transformers(lon: float, lat: float):
    zone = int((lon + 180) / 6) + 1
    epsg = 32600 + zone if lat >= 0 else 32700 + zone
    return (
        Transformer.from_crs(CRS.from_epsg(4326), CRS.from_epsg(epsg), always_xy=True),
        Transformer.from_crs(CRS.from_epsg(epsg), CRS.from_epsg(4326), always_xy=True),
        epsg
    )

def project_geom(g, fwd: Transformer):
    if isinstance(g, Point):
        x,y = fwd.transform(g.x, g.y)
        return Point(x,y)
    elif isinstance(g, LineString):
        pts = [fwd.transform(x,y) for x,y in g.coords]
        return LineString(pts)
    elif isinstance(g, Polygon):
        pts = [fwd.transform(x,y) for x,y in g.exterior.coords]
        return Polygon(pts)
    return g

def min_distance_to_layers(p: Point, layers: List, fwd: Transformer) -> float:
    if not layers: return math.inf
    p_m = project_geom(p, fwd)
    best = math.inf
    for g in layers:
        d = p_m.distance(project_geom(g, fwd))
        if d < best: best = d
    return best

def scenic_score(p: Point, layers: Dict[str, List], weights: Dict[str,float], fwd: Transformer) -> float:
    # Distance scales (meters) set per feature; tweak as needed
    scales = {"water": 120.0, "parks": 75.0}
    score = 0.0

    # land proximity components (water, parks)
    if weights.get("waterfront", 0) > 0 and "water" in layers:
        d = min_distance_to_layers(p, layers["water"], fwd)
        score += weights["waterfront"] * math.exp(- d / scales["water"])
    if weights.get("greenery", 0) > 0 and "parks" in layers:
        d = min_distance_to_layers(p, layers["parks"], fwd)
        score += weights["greenery"] * math.exp(- d / scales["parks"])

    # if the POI is itself a matching category (e.g., viewpoint/cafe), add a bump
    # (Callers can pass tags; here, we just leave a placeholder hook.)
    return score

In [ ]:
# ----------------- Visualization (Folium) -------------------------------------
def add_polygon_layer(m, poly, name, color="#5555ff", fill_color="#9999ff33"):
    gj = folium.GeoJson(mapping(poly), name=name, style_function=lambda x: {
        "color": color, "weight": 2, "fillColor": fill_color, "fillOpacity": 0.25
    })
    gj.add_to(m)

def add_polys_lines_layer(m, geoms, name, color="#00aa66"):
    feat = {
        "type": "FeatureCollection",
        "features": []
    }
    for g in geoms:
        try:
            feat["features"].append({"type":"Feature","geometry":mapping(g),"properties":{}})
        except Exception:
            pass
    folium.GeoJson(feat, name=name, style_function=lambda x: {"color": color, "weight": 2}).add_to(m)

def add_points_layer(m, df: pd.DataFrame, name, icon_color="blue"):
    fg = folium.FeatureGroup(name=name, show=True)
    for _, r in df.iterrows():
        folium.Marker(
            location=[r["lat"], r["lon"]],
            icon=folium.Icon(color=icon_color, icon="info-sign"),
            popup=folium.Popup(html=f"<b>{r.get('name','(unnamed)')}</b><br/>{r.get('category','')}"
                                  f"<br/>score={r.get('score',0):.3f}", max_width=250)
        ).add_to(fg)
    fg.add_to(m)

In [ ]:
# --- ORS route call: origin -> waypoint -> destination in ONE request --------
def ors_route_via_waypoint(olon, olat, wlon, wlat, dlon, dlat,
                           profile="foot-walking",
                           preference="fastest",
                           avoid_features=None,
                           avoid_polygon=None,
                           instructions=False):
    """
    Returns (LineString in lon/lat, distance_m, duration_s).
    """
    url = f"https://api.openrouteservice.org/v2/directions/{profile}/geojson"
    key = os.environ.get("ORS_API_KEY")
    if not key:
        raise RuntimeError("Missing ORS_API_KEY")

    payload = {
        "coordinates": [[olon, olat], [wlon, wlat], [dlon, dlat]],
        "preference": preference,
        "instructions": instructions,
        "units": "m"
    }
    opts = {}
    if avoid_features:
        opts["avoid_features"] = avoid_features
    if avoid_polygon is not None:
        # avoid_polygon should be a Shapely polygon in lon/lat; pass GeoJSON
        from shapely.geometry import mapping
        opts["avoid_polygons"] = mapping(avoid_polygon)
    if opts:
        payload["options"] = opts

    headers = {"Authorization": key, "Content-Type": "application/json"}
    r = requests.post(url, headers=headers, json=payload, timeout=60)
    r.raise_for_status()
    gj = r.json()
    feat = gj["features"][0]
    geom = shape(feat["geometry"])                  # LineString in lon/lat
    summary = feat["properties"]["summary"]
    return geom, summary["distance"], summary["duration"]


def add_route_polyline(m, line_ll: LineString, name, color="#cc0000", weight=6):
    # ORS gives lon,lat; Folium wants lat,lon
    coords_latlon = [(lat, lon) for lon, lat in list(line_ll.coords)]
    folium.PolyLine(coords_latlon, color=color, weight=weight, opacity=0.85,
                    tooltip=name).add_to(m)

In [ ]:
# ----------------- Main Pipeline -------------------------------------
print("Parsing preferences...")
weights = parse_preferences(PREFERENCE_TEXT)
print("Weights:", weights)
routing_opts = parse_routing_constraints(CONSTRAINTS_TEXT)
print("Routing options:", routing_opts)

print("Geocoding...")
olat, olon = geocode_nominatim(ORIGIN_TEXT)
dlat, dlon = geocode_nominatim(DEST_TEXT)

print("Isochrones...")
iso_o = orservice_isochrone(olon, olat, ISO_MIN, PROFILE)
iso_d = orservice_isochrone(dlon, dlat, ISO_MIN, PROFILE)
poly_o = isochrone_polygon(iso_o)
poly_d = isochrone_polygon(iso_d)
search_zone = poly_o.intersection(poly_d)
if search_zone.is_empty:
    raise RuntimeError("Empty intersection; increase ISO_MIN or choose closer O/D.")
bbox = bbox_from_polygon(search_zone)

print("Overpass queries (driven by preferences)...")
queries = build_overpass_multi_query(bbox, weights)
land_geoms, poi_points = [], []

land_layers = {"water": [], "parks": []}
if "land_geoms" in queries:
    j = overpass(queries["land_geoms"])
    geoms, _ = elements_to_geoms(j.get("elements", []))
    # keep only geometries that intersect the polygon
    for g in geoms:
        if g.is_empty: continue
        if g.intersects(search_zone):
            # split water vs parks by a simple tag test heuristic: Overpass JSON keeps tags per element,
            # but we merged geoms—so we can't inspect tags easily at this point.
            # For visualization, just collect all; for scoring, we keep buckets separate by issuing
            # separate queries in a more advanced version. For this demo, we'll assume:
            land_geoms.append(g)
    # Quick split attempt: re-query separately (simpler but more calls) for better buckets:
    # (To keep this demo concise, we skip splitting precisely; scoring still works if we put all into both.)
    land_layers["water"] = [g for g in land_geoms] if weights.get("waterfront",0)>0 else []
    land_layers["parks"] = [g for g in land_geoms] if weights.get("greenery",0)>0 else []

if "poi_points" in queries:
    j = overpass(queries["poi_points"])
    _, pts = elements_to_geoms(j.get("elements", []))
    poi_points = [p for p in pts if search_zone.contains(p)]

# Score candidates
mid_lon, mid_lat = (olon+dlon)/2, (olat+dlat)/2
fwd, rev, epsg = local_metric_transformers(mid_lon, mid_lat)
rows = []
for p in poi_points:
    s = scenic_score(p, land_layers, weights, fwd)
    rows.append({"lat": p.y, "lon": p.x, "score": s, "name": "", "category": ""})
df = pd.DataFrame(rows).sort_values("score", ascending=False).head(TOP_K)

print(f"Found {len(df)} top waypoints.")
if df.empty:
    print("No candidates matched inside the search zone—try broader preferences or bigger isochrones.")

# ----------------- Map -----------------
center_lat, center_lon = (olat+ dlat)/2, (olon+ dlon)/2
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles="cartodbpositron")

# O/D markers
folium.Marker([olat, olon], icon=folium.Icon(color="green"), popup="Origin").add_to(m)
folium.Marker([dlat, dlon], icon=folium.Icon(color="red"), popup="Destination").add_to(m)

# Isochrones + Search zone
add_polygon_layer(m, poly_o, "Origin isochrone", color="#3388ff", fill_color="#3388ff55")
add_polygon_layer(m, poly_d, "Destination isochrone", color="#ff8833", fill_color="#ff883355")
add_polygon_layer(m, search_zone, "Search zone (intersection)", color="#8033ff", fill_color="#8033ff33")

# Land features layer (combined for simplicity)
if land_geoms:
    add_polys_lines_layer(m, land_geoms, "Land features (parks/water)", color="#00aa66")

# Top waypoints
if not df.empty:
    add_points_layer(m, df, f"Top {len(df)} waypoints", icon_color="blue")

# ----------------- Build routes via top-K waypoints & visualize ----------
ROUTE_COLORS = ["#cc0000", "#0077cc", "#2ca02c", "#ff7f0e", "#9467bd", "#17becf", "#e377c2"]
routes = []

if not df.empty:
    print("Routing via top-K waypoints...")
    for idx, r in enumerate(df.itertuples(index=False)):
        wlat, wlon = r.lat, r.lon
        wscore = getattr(r, "score", 0.0)
        try:
            line_ll, dist_m, dur_s = ors_route_via_waypoint(
                olon, olat, wlon, wlat, dlon, dlat,
                profile=routing_opts["profile"],
                preference=routing_opts["preference"],
                avoid_features=routing_opts["avoid_features"],
                avoid_polygon=None,
                instructions=False
            )
            routes.append({
                "idx": idx,
                "waypoint": (wlat, wlon),
                "waypoint_score": float(wscore),
                "line": line_ll,
                "distance_m": float(dist_m),
                "duration_s": float(dur_s)
            })
        except Exception as e:
            print(f"Routing failed for waypoint #{idx+1}: {e}")

    print(f"Built {len(routes)} candidate routes.")

    # Draw each route
    for j, r in enumerate(routes):
        color = ROUTE_COLORS[j % len(ROUTE_COLORS)]
        mins = r["duration_s"] / 60.0
        km = r["distance_m"] / 1000.0
        label = f"Route #{j+1} via waypoint (score={r['waypoint_score']:.2f}) — {km:.2f} km, {mins:.1f} min"
        add_route_polyline(m, r["line"], label, color=color, weight=6)

    # Highlight the "best" route under a simple ETA tolerance
    if routes:
        fastest = min(routes, key=lambda x: x["duration_s"])
        allowed = [r for r in routes if (r["duration_s"] - fastest["duration_s"]) <= ETA_TOLERANCE_MIN * 60]
        if not allowed:
            allowed = [fastest]
        # simple selection: prefer higher waypoint score among allowed (proxy for scenic value)
        winner = max(allowed, key=lambda x: x["waypoint_score"])
        # Build the same mouseover label for the selected route
        sel_mins = winner["duration_s"] / 60.0
        sel_km = winner["distance_m"] / 1000.0
        sel_label = (
            f"Selected route via waypoint "
            f"(score={winner['waypoint_score']:.2f}) — {sel_km:.2f} km, {sel_mins:.1f} min"
        )
        # Draw the selected route thicker & with the detailed tooltip
        add_route_polyline(m, winner["line"], sel_label, color="#000000", weight=8)
        # Mark the selected waypoint with a popup that also shows the same info
        wlat, wlon = winner["waypoint"]
        folium.CircleMarker(
            [wlat, wlon],
            radius=6,
            color="#000000",
            fill=True,
            fill_opacity=0.9,
            popup=folium.Popup(html=sel_label, max_width=280),
        ).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save(MAP_HTML)
print(f"Wrote map: {MAP_HTML}")

Parsing preferences...
Weights: {'waterfront': 0.6, 'greenery': 0.6, 'viewpoint': 0.4, 'cafe': 0.2}
Routing options: {'profile': 'foot-walking', 'preference': 'shortest', 'avoid_features': ['ferries', 'steps']}
Geocoding...
Isochrones...
Overpass queries (driven by preferences)...
Found 10 top waypoints.
Routing via top-K waypoints...
Built 10 candidate routes.
Wrote map: scenic_search_zone_map.html


In [ ]:
from google.colab import files
files.download("scenic_search_zone_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>